In [1]:
# 引入vgg16网络
from tensorflow_vgg import vgg16
from tensorflow_vgg import utils

# 引入mtcnn_findFace
from face_detect_main import mtcnn_findFace
import tensorflow as tf
from scipy import misc
import os
import cv2
import warnings
warnings.filterwarnings('ignore')

E:\Anaconda3\envs\dl\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
E:\Anaconda3\envs\dl\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
E:\Anaconda3\envs\dl\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [4]:
graph = tf.Graph()
with graph.as_default():
    # 构建vgg16网络并加载参数
    vgg = vgg16.Vgg16()
    input_ = tf.keras.layers.Input(
        [224, 224,
         3])  # tf.compat.v1.placeholder(tf.float32, [None, 224, 224, 3])
    with tf.name_scope("content_vgg"):
        vgg.build(input_)


def feature_extraction(imgs):
    path = "./timg.jpg"
    batch = []

    if len(imgs) > 0:
        if imgs.ndim > 3:
            for i in range(imgs.shape[0]):
                cv2.imwrite("./face_feature.png", imgs[i])
                # 调用开源代码实现的函数load_image加载人脸图像
                img = utils.load_image(path)
                batch.append(img)
        else:

            cv2.imwrite("./face_feature.png", imgs)
            img = utils.load_image(path)
            batch.append(img.reshape((1, 224, 224, 3)))
            batch = np.concatenate(batch)

        with tf.compat.v1.Session(graph=graph) as sess:
            feed_dict = {input_: batch}
            # 前向传播获得vgg.fc7(就是第二个全连接层)的输出作为人脸特征
            feature = sess.run(vgg.fc7, feed_dict=feed_dict)
            if imgs.ndim == 3:
                feature = np.reshape(feature, (1, 4096))
        # 返回人脸特征
        return feature

D:\Program Files\Pyproject\PS\Face_Recognizer-master\Face_Recognizer-upgradedToTF2.0\tensorflow_vgg\vgg16.npy
npy file loaded
build model started
build model finished: 4s


In [5]:
if __name__ == '__main__':
    image_path = "./timg.jpg"
    img = misc.imread(os.path.expanduser(image_path), mode='RGB')
    images, rects = mtcnn_findFace(img)
    feature_extraction(images)